## Install Libraries

We will install the software libraries necessary for this week's tutorials, and test them to make sure they work.

You can try to install all  required componenets by running the comand below.

If that does not work you can slo use command-line or terminal. Have alook at the linked page for more detail. [the linked wiki page will help you](https://github.com/DavidGoedicke/RealtimeAudioClassification/wiki/Command-lines-and-terminals-a-quick-introduction.md)

The libraries that we will use.

[NumPy](https://en.wikipedia.org/wiki/NumPy) (Python support for large, multi-dimensional arrays and matrices ).

[MatplotLib](https://matplotlib.org) (Python 2D plotting library).

[pyTorch](https://pytorch.org/get-started/locally/) (Python Machine Learning Library).

[sklearn](https://scikit-learn.org/stable/) (SciKit Machine Learning library).

[librosa](https://librosa.github.io/librosa/) (Music and Audio analysis library).

[PyAudio](https://people.csail.mit.edu/hubert/pyaudio/) (Python bindings for PortAudio, the cross-platform audio I/O library).


In [1]:
pip install numpy matplotlib torch torchvision scikit-learn librosa pyaudio numpy_ringbuffer ipywidgets

Processing c:\users\farla\appdata\local\pip\cache\wheels\aa\5a\92\d52f6f8560ff05a2525e6030a1903412df876714241fb76802\librosa-0.8.0-py3-none-any.whl
  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  Using cached numpy_ringbuffer-0.2.1-py2.py3-none-any.whl (3.8 kB)
Processing c:\users\farla\appdata\local\pip\cache\wheels\6f\d1\5d\f13da53b1dcbc2624ff548456c9ffb526c914f53c12c318bb4\resampy-0.2.2-py3-none-any.whl
Processing c:\users\farla\appdata\local\pip\cache\wheels\9b\94\80\3673f65684ab97e08999d1460fc1b238df7701805c739791cd\audioread-2.1.8-py3-none-any.whl
  Using cached SoundFile-0.10.3.post1-py2.py3.cp26.cp27.cp32.cp33.cp34.cp35.cp36.pp27.pp32.pp33-none-win_amd64.whl (689 kB)
  Using cached pooch-1.1.1-py3-none-any.whl (45 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio: started
    Running setup.py install for pyaudio: finished with status 'error'
Note: you may need to re

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\farla\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\farla\\AppData\\Local\\Temp\\pip-install-78o7alwu\\pyaudio\\setup.py'"'"'; __file__='"'"'C:\\Users\\farla\\AppData\\Local\\Temp\\pip-install-78o7alwu\\pyaudio\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\farla\AppData\Local\Temp\pip-wheel-y0gylh73'
       cwd: C:\Users\farla\AppData\Local\Temp\pip-install-78o7alwu\pyaudio\
  Complete output (17 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  copying src\pyaudio.py -> build\lib.win-amd64-3.8
  running build_ext
  building '_portaudio' extension
  creating build\temp.win-amd64-3.8
  creating build\temp.win-amd64-3.8\Release
  creating build\temp.wi

## Test setup

In the code below, we load ("import") the libraries that we just installed above, to make sure that Python can find the libraries. 


In [1]:
import csv
import os
import wave
import time
import multiprocessing
from threading import Thread
import os
print("-------Found all built-in libraries.-------")
import numpy as np
print("Found numpy ",np.__version__)
import matplotlib
print("Found matplotlib ",matplotlib.__version__)
print("-------Found all math libraries.-------")
import pyaudio
print("Found pyaudio ",pyaudio.__version__)
import librosa
print("Found librosa ",librosa.__version__)
print("-------Found all audio libraries.-------")
import torch
print("Found torch ",torch.__version__)
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix
print("-------Found all machine learning libraries.-------")


-------Found all built-in libraries.-------
Found numpy  1.18.5
Found matplotlib  3.2.2
-------Found all math libraries.-------
Found pyaudio  0.2.11
Found librosa  0.8.0
-------Found all audio libraries.-------
Found torch  1.6.0
-------Found all machine learning libraries.-------


If the above code runs without a problem, all libraries have installed and can be loaded. While it does not gurantee that everything will work, this is a very good sign.
